<a href="https://colab.research.google.com/github/Pdaret/Steel-Industry-Energy-Consumption/blob/main/Steel_Industry_Energy_consumption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[Steel Industry Energy Consumption](https://www.kaggle.com/datasets/csafrit2/steel-industry-energy-consumption)**
### *<font color='blue'>Energy Usage prediction</font>

 in this project we want to train a model to predict the <font color='green'>Energy Usage</font> of Steel Industry Based on its Dataset.

# Data Preparation

outlier

In [29]:
import pandas as pd
import plotly.express as px
from google.colab import drive
drive.mount('/content/drive')
from sklearn.preprocessing import LabelEncoder
import scipy
from scipy import stats

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data Mining Project/Steel_industry_data.csv')
# df = pd.read_csv('Steel_industry_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35040 entries, 0 to 35039
Data columns (total 11 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   date                                  35040 non-null  object 
 1   Usage_kWh                             35040 non-null  float64
 2   Lagging_Current_Reactive.Power_kVarh  35040 non-null  float64
 3   Leading_Current_Reactive_Power_kVarh  35040 non-null  float64
 4   CO2(tCO2)                             35040 non-null  float64
 5   Lagging_Current_Power_Factor          35040 non-null  float64
 6   Leading_Current_Power_Factor          35040 non-null  float64
 7   NSM                                   35040 non-null  int64  
 8   WeekStatus                            35040 non-null  object 
 9   Day_of_week                           35040 non-null  object 
 10  Load_Type                             35040 non-null  object 
dtypes: float64(6), 

In [6]:
df = df.rename(columns = {'Lagging_Current_Reactive.Power_kVarh': 'Lagging_Current_Reactive_Power_kVarh'
                ,'CO2(tCO2)':'CO2'})

In [7]:
df.columns

Index(['date', 'Usage_kWh', 'Lagging_Current_Reactive_Power_kVarh',
       'Leading_Current_Reactive_Power_kVarh', 'CO2',
       'Lagging_Current_Power_Factor', 'Leading_Current_Power_Factor', 'NSM',
       'WeekStatus', 'Day_of_week', 'Load_Type'],
      dtype='object')

In [8]:
df.describe()[['Usage_kWh', 'Lagging_Current_Reactive_Power_kVarh','Leading_Current_Reactive_Power_kVarh']]

,Usage_kWh,Lagging_Current_Reactive_Power_kVarh,Leading_Current_Reactive_Power_kVarh
count,35040.000000,35040.000000,35040.000000
mean,27.386892,13.035384,3.870949
std,33.444380,16.306000,7.424463
min,0.000000,0.000000,0.000000
25%,3.200000,2.300000,0.000000
50%,4.570000,5.000000,0.000000
75%,51.237500,22.640000,2.090000
max,157.180000,96.910000,27.760000


In [9]:
fig1 = px.histogram(df, x='Usage_kWh')
#drawing histogram for Usage
fig1.show()

In [10]:
fig2 = px.box(df, y='Usage_kWh')
#drawing boxplot for Usage
fig2.show()

In [11]:
Q1=df.Usage_kWh.quantile(0.25)
Q3=df.Usage_kWh.quantile(0.75)

In [12]:
IQR=Q3-Q1
IQR

48.037499999999994

In [13]:
lower_limit=Q1-1.5*IQR
upper_limit=Q3+1.5*IQR
lower_limit,upper_limit

(-68.85624999999999, 123.29374999999999)

In [14]:
#outliers
df[(df.Usage_kWh<lower_limit)|(df.Usage_kWh>upper_limit)]

,date,Usage_kWh,Lagging_Current_Reactive_Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2,Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
140,02/01/2018 11:15,128.52,70.85,0.0,0.00,87.57,100.0,40500,Weekday,Tuesday,Maximum_Load
141,02/01/2018 11:30,144.47,78.48,0.0,0.00,87.87,100.0,41400,Weekday,Tuesday,Maximum_Load
143,02/01/2018 12:00,123.30,66.06,0.0,0.00,88.15,100.0,43200,Weekday,Tuesday,Maximum_Load
150,02/01/2018 13:45,124.45,49.79,0.0,0.00,92.85,100.0,49500,Weekday,Tuesday,Medium_Load
153,02/01/2018 14:30,129.10,53.50,0.0,0.00,92.38,100.0,52200,Weekday,Tuesday,Medium_Load
...,...,...,...,...,...,...,...,...,...,...,...
33851,19/12/2018 15:00,145.22,70.67,0.0,0.07,89.92,100.0,54000,Weekday,Wednesday,Medium_Load
33853,19/12/2018 15:30,130.46,60.23,0.0,0.06,90.79,100.0,55800,Weekday,Wednesday,Medium_Load
33855,19/12/2018 16:00,125.21,58.43,0.0,0.06,90.62,100.0,57600,Weekday,Wednesday,Medium_Load
33856,19/12/2018 16:15,133.56,64.40,0.0,0.06,90.08,100.0,58500,Weekday,Wednesday,Medium_Load


In [15]:
fig3 = px.histogram(df, x='Lagging_Current_Reactive_Power_kVarh')
fig3.show()
#drawing histogram for Lagging_Current_Reactive.Power_kVarh

In [16]:
fig4 = px.box(df, y='Lagging_Current_Reactive_Power_kVarh')
fig4.show()
#drawing boxplot for Lagging_Current_Reactive.Power_kVarh

In [17]:
Q1=df.Lagging_Current_Reactive_Power_kVarh.quantile(0.25)
Q3=df.Lagging_Current_Reactive_Power_kVarh.quantile(0.75)

In [18]:
IQR=Q3-Q1
IQR

20.34

In [19]:
lower_limit=Q1-1.5*IQR
upper_limit=Q3+1.5*IQR

In [20]:
df[(df.Lagging_Current_Reactive_Power_kVarh<lower_limit)|(df.Lagging_Current_Reactive_Power_kVarh>upper_limit)]
#outliers

,date,Usage_kWh,Lagging_Current_Reactive_Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2,Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
140,02/01/2018 11:15,128.52,70.85,0.0,0.00,87.57,100.0,40500,Weekday,Tuesday,Maximum_Load
141,02/01/2018 11:30,144.47,78.48,0.0,0.00,87.87,100.0,41400,Weekday,Tuesday,Maximum_Load
142,02/01/2018 11:45,90.86,54.29,0.0,0.00,85.84,100.0,42300,Weekday,Tuesday,Maximum_Load
143,02/01/2018 12:00,123.30,66.06,0.0,0.00,88.15,100.0,43200,Weekday,Tuesday,Maximum_Load
152,02/01/2018 14:15,123.12,55.26,0.0,0.00,91.23,100.0,51300,Weekday,Tuesday,Medium_Load
...,...,...,...,...,...,...,...,...,...,...,...
33853,19/12/2018 15:30,130.46,60.23,0.0,0.06,90.79,100.0,55800,Weekday,Wednesday,Medium_Load
33855,19/12/2018 16:00,125.21,58.43,0.0,0.06,90.62,100.0,57600,Weekday,Wednesday,Medium_Load
33856,19/12/2018 16:15,133.56,64.40,0.0,0.06,90.08,100.0,58500,Weekday,Wednesday,Medium_Load
33921,20/12/2018 08:30,79.85,60.98,0.0,0.04,79.48,100.0,30600,Weekday,Thursday,Light_Load


In [21]:
fig5 = px.histogram(df, x='Leading_Current_Reactive_Power_kVarh')
fig5.show()
#drawing histogram for Leading_Current_Reactive_Power_kVarh

In [22]:
fig6 = px.box(df, y='Leading_Current_Reactive_Power_kVarh')
fig6.show()
#drawing boxplot for Leading_Current_Reactive_Power_kVarh

In [23]:
Q1=df.Leading_Current_Reactive_Power_kVarh.quantile(0.25)
Q3=df.Leading_Current_Reactive_Power_kVarh.quantile(0.75)

In [24]:
IQR=Q3-Q1

In [25]:
lower_limit=Q1-1.5*IQR
upper_limit=Q3+1.5*IQR

In [26]:
df[(df.Leading_Current_Reactive_Power_kVarh<lower_limit)|(df.Leading_Current_Reactive_Power_kVarh>upper_limit)]
#outliers

,date,Usage_kWh,Lagging_Current_Reactive_Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2,Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
35,01/01/2018 09:00,3.89,0.0,15.66,0.0,100.0,24.11,32400,Weekday,Monday,Light_Load
36,01/01/2018 09:15,3.60,0.0,16.96,0.0,100.0,20.76,33300,Weekday,Monday,Light_Load
37,01/01/2018 09:30,4.03,0.0,21.28,0.0,100.0,18.61,34200,Weekday,Monday,Light_Load
38,01/01/2018 09:45,3.92,0.0,19.22,0.0,100.0,19.98,35100,Weekday,Monday,Light_Load
39,01/01/2018 10:00,3.64,0.0,16.88,0.0,100.0,21.08,36000,Weekday,Monday,Light_Load
...,...,...,...,...,...,...,...,...,...,...,...
35027,31/12/2018 21:00,3.42,0.0,13.36,0.0,100.0,24.80,75600,Weekday,Monday,Light_Load
35028,31/12/2018 21:15,3.42,0.0,13.39,0.0,100.0,24.75,76500,Weekday,Monday,Light_Load
35029,31/12/2018 21:30,3.49,0.0,13.36,0.0,100.0,25.27,77400,Weekday,Monday,Light_Load
35030,31/12/2018 21:45,3.42,0.0,13.36,0.0,100.0,24.80,78300,Weekday,Monday,Light_Load


 **Standardizing**

In [31]:
#standardizing
review = pd.DataFrame(df)
review
review['Usage_kWh']=stats.zscore(review['Usage_kWh'])
review['Lagging_Current_Reactive_Power_kVarh']=stats.zscore(review['Lagging_Current_Reactive_Power_kVarh'])
review['Leading_Current_Reactive_Power_kVarh']=stats.zscore(review['Leading_Current_Reactive_Power_kVarh'])
review['CO2']=stats.zscore(review['CO2'])
review['Lagging_Current_Power_Factor']=stats.zscore(review['Lagging_Current_Power_Factor'])
review['Leading_Current_Power_Factor']=stats.zscore(review['Leading_Current_Power_Factor'])
review['NSM']=stats.zscore(review['NSM'])
review
review.to_csv('standared.csv',index=False)
#file after standardalize 

Transforming

In [32]:
#transforming

le=LabelEncoder()
label=le.fit_transform(df['WeekStatus'])
df.drop("WeekStatus",axis=1,inplace=True)
df["WeekStatus"]=label
df
#transform with label encoder
#weekday->0  weekend->1

,date,Usage_kWh,Lagging_Current_Reactive_Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2,Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,Day_of_week,Load_Type,WeekStatus
0,01/01/2018 00:15,-0.724105,-0.618516,-0.521385,-0.71355,-0.389410,0.513268,-1.678015,Monday,Light_Load,0
1,01/01/2018 00:30,-0.699287,-0.525911,-0.521385,-0.71355,-0.729772,0.513268,-1.641929,Monday,Light_Load,0
2,01/01/2018 00:45,-0.722012,-0.598278,-0.521385,-0.71355,-0.544264,0.513268,-1.605843,Monday,Light_Load,0
3,01/01/2018 01:00,-0.719919,-0.581106,-0.521385,-0.71355,-0.660009,0.513268,-1.569756,Monday,Light_Load,0
4,01/01/2018 01:15,-0.704669,-0.523458,-0.521385,-0.71355,-0.838117,0.513268,-1.533670,Monday,Light_Load,0
...,...,...,...,...,...,...,...,...,...,...,...
35035,31/12/2018 23:00,-0.703772,-0.501380,-0.521385,-0.71355,-0.976587,0.513268,1.605843,Monday,Light_Load,0
35036,31/12/2018 23:15,-0.707061,-0.570067,-0.521385,-0.71355,-0.521538,0.513268,1.641929,Monday,Light_Load,0
35037,31/12/2018 23:30,-0.705865,-0.605024,-0.511957,-0.71355,-0.209188,0.512611,1.678015,Monday,Light_Load,0
35038,31/12/2018 23:45,-0.705865,-0.611770,-0.506569,-0.71355,-0.151052,0.511954,1.714102,Monday,Light_Load,0


In [33]:
one_hot = pd.get_dummies(df['Day_of_week'])
df = df.drop('Day_of_week',axis = 1)
df = df.join(one_hot)
df
#transform with one_hot for day_of_week

,date,Usage_kWh,Lagging_Current_Reactive_Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2,Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,Load_Type,WeekStatus,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,01/01/2018 00:15,-0.724105,-0.618516,-0.521385,-0.71355,-0.389410,0.513268,-1.678015,Light_Load,0,0,1,0,0,0,0,0
1,01/01/2018 00:30,-0.699287,-0.525911,-0.521385,-0.71355,-0.729772,0.513268,-1.641929,Light_Load,0,0,1,0,0,0,0,0
2,01/01/2018 00:45,-0.722012,-0.598278,-0.521385,-0.71355,-0.544264,0.513268,-1.605843,Light_Load,0,0,1,0,0,0,0,0
3,01/01/2018 01:00,-0.719919,-0.581106,-0.521385,-0.71355,-0.660009,0.513268,-1.569756,Light_Load,0,0,1,0,0,0,0,0
4,01/01/2018 01:15,-0.704669,-0.523458,-0.521385,-0.71355,-0.838117,0.513268,-1.533670,Light_Load,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35035,31/12/2018 23:00,-0.703772,-0.501380,-0.521385,-0.71355,-0.976587,0.513268,1.605843,Light_Load,0,0,1,0,0,0,0,0
35036,31/12/2018 23:15,-0.707061,-0.570067,-0.521385,-0.71355,-0.521538,0.513268,1.641929,Light_Load,0,0,1,0,0,0,0,0
35037,31/12/2018 23:30,-0.705865,-0.605024,-0.511957,-0.71355,-0.209188,0.512611,1.678015,Light_Load,0,0,1,0,0,0,0,0
35038,31/12/2018 23:45,-0.705865,-0.611770,-0.506569,-0.71355,-0.151052,0.511954,1.714102,Light_Load,0,0,1,0,0,0,0,0


In [34]:
one_hot = pd.get_dummies(df['Load_Type'])
df = df.drop('Load_Type',axis = 1)
df = df.join(one_hot)
df
#transform with one_hot for load_type

,date,Usage_kWh,Lagging_Current_Reactive_Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2,Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Light_Load,Maximum_Load,Medium_Load
0,01/01/2018 00:15,-0.724105,-0.618516,-0.521385,-0.71355,-0.389410,0.513268,-1.678015,0,0,1,0,0,0,0,0,1,0,0
1,01/01/2018 00:30,-0.699287,-0.525911,-0.521385,-0.71355,-0.729772,0.513268,-1.641929,0,0,1,0,0,0,0,0,1,0,0
2,01/01/2018 00:45,-0.722012,-0.598278,-0.521385,-0.71355,-0.544264,0.513268,-1.605843,0,0,1,0,0,0,0,0,1,0,0
3,01/01/2018 01:00,-0.719919,-0.581106,-0.521385,-0.71355,-0.660009,0.513268,-1.569756,0,0,1,0,0,0,0,0,1,0,0
4,01/01/2018 01:15,-0.704669,-0.523458,-0.521385,-0.71355,-0.838117,0.513268,-1.533670,0,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35035,31/12/2018 23:00,-0.703772,-0.501380,-0.521385,-0.71355,-0.976587,0.513268,1.605843,0,0,1,0,0,0,0,0,1,0,0
35036,31/12/2018 23:15,-0.707061,-0.570067,-0.521385,-0.71355,-0.521538,0.513268,1.641929,0,0,1,0,0,0,0,0,1,0,0
35037,31/12/2018 23:30,-0.705865,-0.605024,-0.511957,-0.71355,-0.209188,0.512611,1.678015,0,0,1,0,0,0,0,0,1,0,0
35038,31/12/2018 23:45,-0.705865,-0.611770,-0.506569,-0.71355,-0.151052,0.511954,1.714102,0,0,1,0,0,0,0,0,1,0,0


In [36]:
review=pd.DataFrame(df)
review
review.to_csv('transformed.csv',index=False)

# Exploratory Data Analysis

# Pre-Model 

# Modeling (Linear Regression)

# Model Evaluation